<a href="https://colab.research.google.com/github/paulandrepamm/proyecto_etfs_eu/blob/main/Copia_de_Proyecto_1_Hack_A_Boss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

# Proyecto 1: Hack a Boss
## Análisis del Top 10 ETFs

Proyecto de análisis del Top 10 de ETFs.

### Documentación del Proyecto

1. [Presentación Paula](https://docs.google.com/presentation/d/1Wk4tVhK89EP7b4iiIhvcTk4n5uHJkxgnuADb6Dk_Nto/edit?usp=sharing)

### Páginas datos fundamentales
1. [MorningStar](https://www.morningstar.es/es/)
2. [Dataroma](https://www.dataroma.com/m/home.php)

---





# Google Drive
## Esta celda monta **Google Drive** para que podamos guardar los archivos csv y cualquier otro archivo que se vaya generando.
### IMPORTANTE (Se crea y se monta la carpeta *Bootcamp_Proyecto1* en **vuestro** Google Drive)

In [ ]:
import os
from google.colab import drive

# Se monta la carpeta principal de tu Google Drive
drive.mount('/content/drive', force_remount=True)
# Se define la carpeta de trabajo
carpeta_trabajo = '/content/drive/My Drive/Bootcamp_Proyecto1/archivos'
# Se comprueba si existe, si no, se crea
if not os.path.exists(carpeta_trabajo):
    os.makedirs(carpeta_trabajo, exist_ok=True)
# Se establece la carpeta como directorio de trabajo
os.chdir(carpeta_trabajo)
# Comprobación
print(f"El sistema se encuentra en {os.getcwd()}")

# Importación de **librerías**
### Incluid aquí las librerías a instalar e importar

In [ ]:
# Instalar las librerías
!pip install yfinance
!pip install yahooquery


In [34]:
# Importar las librerías
import time
import requests
import warnings
import numpy as np
import yfinance as yf
import yahooquery as yq
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from pprint import pprint
warnings.filterwarnings("ignore")

# Sección Extracción de datos (API y WebScrapin)





In [62]:
def seleccion_etf() -> list:
    # Obtención de los símbolos del Top ETFs de EEUU
    s = yq.Screener()
    dict_query = s.get_screeners(['top_etfs_us'])
    tickers = [symbol['symbol'] for symbol in dict_query['top_etfs_us']['quotes']]
    return tickers

def corregir_nombres_columnas(df:pd.DataFrame) -> pd.DataFrame:
    # Eliminar caracteres especiales
    df.columns = df.columns.str.replace('[^\w\s]', '', regex=True)
    # Sustituir espacios por _
    df.columns = df.columns.str.replace(' ', '_')
    # Retornar el DF
    return(df)
    
def down_price_vol(tickers:list) -> pd.DataFrame:
    '''
    Esta función descarga los cambios porcentuales de las cotizaciones
    y del volumen de negociación de la lista de símbolos en un dataframe.

    Parámetros:
    tickers (list): La lista de símbolos.

    Retorna:
    DataFrame con los incrementos porcentuales de las cotizaciones

    Ejemplo:
    >>> df_price_vol = down_price_vol(['MSFT'])
    '''
    # Descarga de las cotizaciones
    df = yf.download(tickers, period='max')
    # Obtener los nombres de las columnas para 'Adj Close' y 'Volume'
    adj_close_cols = [('Adj Close', ticker) for ticker in tickers]
    volume_cols = [('Volume', ticker) for ticker in tickers]

    # Seleccionar las columnas de interés
    df = df[adj_close_cols + volume_cols]
    # Eliminar valores nulos y cambiar los precios por incrementos porcentuales
    df = df.dropna().pct_change(1).dropna()
    # Ordenar y formatear el dataframe
    df_stack = df.stack(level=1)
    df_reset = df_stack.reset_index()
    df_final = df_reset.rename(columns={'level_0': 'Date', 'level_1': 'Ticker'})
    df_final = df_final.melt(id_vars=['Date', 'Ticker'], var_name='Attribute', value_name='Value')
    df_pivot = df_final.pivot_table(index=['Date', 'Ticker'], columns='Attribute', values='Value').reset_index()
    df_pivot.columns.name = None
    df = df_pivot.rename(columns={'Adj Close': 'Price', 'Volume': 'Volume'})
    # Retornar DataFrame
    return(df)

def web_scraping_sectors(tickers:list) -> pd.DataFrame:
    '''
    Esta función hace web scraping a la página de Yahoo Finance
    para obtener la distribución en los sectores en los que está
    invertido el ETF

    Parámetros:
    tickers (list): La lista de símbolos.

    Retorna:
    DataFrame con los porcentajes de la distribución de sectores

    Ejemplo:
    >>> df_sectores = web_scraping_sectores()
    '''
    url_base = "https://es.finance.yahoo.com/quote/"
    lista_sectores = [
        "Materiales básicos", "Acciones cíclicas", "Servicios financieros",
        "Propiedades inmobiliarias", "Acciones defensivas", "Atención sanitaria",
        "Utilidades", "Servicios de comunicación", "Energía", "Industriales",
        "Tecnología"
    ]
    dict_sectores = {}
    for etf in tickers:
        url = f'{url_base}{etf}/holdings?p={etf}'
        response = requests.get(
            url,
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
            )
        soup = BeautifulSoup(response.text, 'html.parser')
        sectores_etf = {}
        lista_contenedores = soup.findAll('div', class_='Bdbw(1px) Bdbc($seperatorColor) Bdbs(s) H(25px) Pt(10px)')
        for item in lista_contenedores:
            sector = item.find('span', class_='Mend(5px) Whs(nw)').text
            if sector in lista_sectores:
                valor = item.find('span', class_='W(20%) D(b) Fl(start) Ta(e)').text
                sectores_etf[sector] = valor
        dict_sectores[etf] = sectores_etf
        time.sleep(0.5)
    # Convertir diccionario en dataframe
    df = pd.DataFrame.from_dict(dict_sectores).T
    # Corregir nombres de columnas
    df = corregir_nombres_columnas(df)
    # Convertir todas las entradas a strings, eliminar el '%' y reemplazar comas por puntos
    df = df.map(lambda x: str(x).replace('%', '').replace(',', '.'))
    # Convertir las cadenas a números flotantes y dividir por 100 para tener los porcentajes en formato decimal
    df = df.map(lambda x: float(x) / 100.0)
    # Retornar DataFrame
    return (df)

def tabla_net_assets(tickers:list) -> pd.DataFrame:
    def convertir_valor(valor):
        if 'M' in valor:
            return float(valor.replace('M', ''))
        elif 'B' in valor:
            return float(valor.replace('B', '')) * 1000
        else:
            return float(valor)

    #Net assets table

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
    }

    net_assets = pd.DataFrame()

    for ticker in tickers:

        url = f"https://finance.yahoo.com/quote/{ticker}?p={ticker}"
        response = requests.get(url, headers=headers)

        try:
            tables = pd.read_html(response.text)
            for table in tables:
                labels_to_remove = ["Previous Close", "Open", "Bid", "Ask", "Day's Range", "52 Week Range", "Volume","Avg. Volume"]
                table = table[~table.iloc[:, 0].isin(labels_to_remove)]

                data_series = pd.Series(dict(zip(table.iloc[:, 0], table.iloc[:, 1])))

                net_assets[ticker] = data_series
        except ValueError:
            print(f"No hay tablas para {ticker}")

    net_assets = net_assets.T
    # Eliminar columnas
    net_assets = net_assets.drop(['NAV', 'PE Ratio (TTM)', 'Expense Ratio (net)'], axis=1)
    # Corregir nombres de columnas
    df = corregir_nombres_columnas(net_assets)
    # Convertir todas las entradas a strings, eliminar el '%' y reemplazar comas por puntos
    df[['Yield', 'YTD_Daily_Total_Return']] = df[['Yield', 'YTD_Daily_Total_Return']].map(lambda x: str(x).replace('%', ''))
    # Convertir las cadenas a números flotantes y dividir por 100 para tener los porcentajes en formato decimal
    df[['Yield', 'YTD_Daily_Total_Return']] = df[['Yield', 'YTD_Daily_Total_Return']].map(lambda x: float(x) / 100.0)
    # Convertir la columna Beta a float
    df['Beta_5Y_Monthly'] = df['Beta_5Y_Monthly'].astype(float)
    # Convertir Inception_Date en datetime
    df['Inception_Date'] = pd.to_datetime(df['Inception_Date'], format='%Y-%m-%d')
    # Reformatear columna Net_Assets
    df['Net_Assets_Millions'] = df['Net_Assets'].apply(convertir_valor)
    df = df.drop('Net_Assets', axis=1)
    # Retornar df
    return(df)

def portafolio_activos_etfs(tickers:list)->pd.DataFrame:
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
    }

    all_dataframes = []

    for ticker in tickers:

        url = f"https://finance.yahoo.com/quote/{ticker}/holdings?p={ticker}"
        response = requests.get(url, headers=headers)

        try:
            tables = pd.read_html(response.text)
            for table in tables:
                table['Ticker'] = ticker  # Add the 'Ticker' column
                all_dataframes.append(table)
        except ValueError:
            print(f"No tables found for {ticker}")

    # Concatenate all dataframes
    merged_df = pd.concat(all_dataframes, ignore_index=True)
    # Corregir nombres de columnas
    df = corregir_nombres_columnas(merged_df)
    # Sustituir espacios por _
    df.columns = df.columns.str.replace('_', '')
    # Convertir todas las entradas a strings, eliminar el '%' y reemplazar comas por puntos
    df['Assets'] = df['Assets'].map(lambda x: str(x).replace('%', ''))
    # Convertir las cadenas a números flotantes y dividir por 100 para tener los porcentajes en formato decimal
    df['Assets'] = df['Assets'].map(lambda x: float(x) / 100.0)
    # Retornar df
    return(df)

In [63]:
tickers = seleccion_etf()
df_price_vol = down_price_vol(tickers)
df_sect_dist = web_scraping_sectors(tickers)
df_net_activos = tabla_net_assets(tickers)
df_portafolio_activos = portafolio_activos_etfs(tickers)

,Name,Symbol,Assets,Ticker
0,Kobe Steel Ltd,5406.T,0.0175,DXJS
1,Cosmo Energy Holdings Co Ltd,5021.T,0.0092,DXJS
2,Tokyo Seimitsu Co Ltd,7729.T,0.0069,DXJS
3,Aozora Bank Ltd,8304.T,0.0068,DXJS
4,Seven Bank Ltd,8410.T,0.0064,DXJS
...,...,...,...,...
228,Sumitomo Mitsui Financial Group Inc,8316.T,0.0167,GSJY
229,Mitsui & Co Ltd,8031.T,0.0166,GSJY
230,Keyence Corp,6861.T,0.0161,GSJY
231,Takeda Pharmaceutical Co Ltd,4502.T,0.0159,GSJY


In [56]:
df_net_activos.dtypes

Yield                            float64
YTD_Daily_Total_Return           float64
Beta_5Y_Monthly                  float64
Inception_Date            datetime64[ns]
Net_Assets_Millions              float64
dtype: object

# Carga de datos a Airtable

In [ ]:
token_id = "patyBg6zyoW4EXNI7"

# Sección Karlos

# Sección Josep

In [ ]:
df


In [ ]:
df_describe = df.describe()
df_describe
